In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

In [22]:
# res_block = 3*3 conv + 1*1 conv + input

#方法一： 原生写法
in_channels = 2
ou_channles = 2
kernel_size = 3
w = 9
h = 9
time1 = time()
x = torch.ones(1,in_channels,w,h)#输入图片
conv_2d = nn.Conv2d(in_channels,ou_channles,kernel_size,padding='same')
conv_2d_pointwise = nn.Conv2d(in_channels,ou_channles,1)
result1 = conv_2d(x) + conv_2d_pointwise(x) + x
time2 = time()
conv_2d_pointwise.weight.shape


torch.Size([2, 2, 1, 1])

In [15]:
# 方法二：算子融合，把point wise卷积与x本身都写成3*3卷积
# 最终把三个卷积写成一个卷积
# 1）改造
pointwise_to_conv_weight = F.pad(conv_2d_pointwise.weight,[1,1,1,1,0,0,0,0])#2*2*1*1 -> 2*2*3*3
conv_2d_for_pointwise = nn.Conv2d(in_channels,ou_channles,kernel_size,padding='same')
conv_2d_for_pointwise.weight = nn.Parameter(pointwise_to_conv_weight)
conv_2d_for_pointwise.bias = conv_2d_pointwise.bias
 
zeros = torch.unsqueeze(torch.zeros(kernel_size, kernel_size),0)
stars = torch.unsqueeze(F.pad(torch.ones(1,1), [1,1,1,1]),0)
stars_zeros = torch.unsqueeze(torch.cat([stars,zeros],0),0)
zeros_stars = torch.unsqueeze(torch.cat([stars,zeros],0),0)#torch.unsqueeze(torch.cat([zeros,stars],0),0)
identity_to_conv_weight = torch.cat([stars_zeros,zeros_stars],0)
identity_to_conv_bias = torch.zeros([ou_channles])
conv_2d_for_identity = nn.Conv2d(in_channels,ou_channles,kernel_size,padding='same')
conv_2d_for_identity.weight = nn.Parameter(identity_to_conv_weight)
conv_2d_for_identity.bias = nn.Parameter(identity_to_conv_bias)
result2 = conv_2d(x) + conv_2d_for_pointwise(x) + conv_2d_for_identity(x)
print(torch.isclose(result1,result2))#保存的是浮点数，不能直接用==判断

tensor([[[[True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True]],

         [[True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
        

In [23]:
# 2） 融合
time3 = time()
conv_2d_for_fusion = nn.Conv2d(in_channels,ou_channles,kernel_size,padding='same')
conv_2d_for_fusion.weight = nn.Parameter(conv_2d.weight.data + conv_2d_for_pointwise.weight.data + conv_2d_for_identity.weight.data)
conv_2d_for_fusion.bias = nn.Parameter(conv_2d.bias.data + conv_2d_for_pointwise.bias.data + conv_2d_for_identity.bias.data)
result3 = conv_2d_for_fusion(x)
time4 = time()
print(torch.all(torch.isclose(result2,result3)))

tensor(False)


In [24]:
print(time2-time1,time4-time3)# 可以看到确实快很多。

0.002000093460083008 0.0009984970092773438
